In [1]:
import sqlite3
import pandas as pd
import os

# 1. Tentukan Lokasi Database
# Asumsi file .ipynb ini ada di folder yang SAMA dengan violations.db
db_path = "violations.db"

# Cek apakah file ada
if not os.path.exists(db_path):
    print(f"❌ File database tidak ditemukan di: {os.path.abspath(db_path)}")
    print("Pastikan file .ipynb ini berada di folder: backend_fastapi")
else:
    print(f"✅ Database ditemukan: {os.path.abspath(db_path)}")
    
    # Buat Koneksi
    conn = sqlite3.connect(db_path)
    print("🔌 Koneksi berhasil!")

✅ Database ditemukan: c:\_workspace\skripsi\results\sinte\folder-aman\project_helm_detection\backend_fastapi\violations.db
🔌 Koneksi berhasil!


In [2]:
# Ambil daftar semua tabel
query_tables = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
tables = pd.read_sql(query_tables, conn)

print(f"📊 Ditemukan {len(tables)} Tabel:\n")

# Loop untuk detail setiap tabel
for index, row in tables.iterrows():
    table_name = row['name']
    print(f"{'='*40}")
    print(f"📂 TABEL: {table_name}")
    print(f"{'='*40}")
    
    # Ambil Info Kolom (PRAGMA)
    # cid = Column ID, type = Tipe Data, pk = Primary Key (1=Ya)
    schema_query = f"PRAGMA table_info({table_name})"
    schema_df = pd.read_sql(schema_query, conn)
    
    # Rapikan tampilan
    display_df = schema_df[['cid', 'name', 'type', 'notnull', 'pk']]
    display_df.columns = ['ID', 'Nama Kolom', 'Tipe Data', 'Wajib Isi?', 'PK?']
    
    # Tampilkan Dataframe
    display(display_df)
    print("\n")

📊 Ditemukan 1 Tabel:

📂 TABEL: violation


,ID,Nama Kolom,Tipe Data,Wajib Isi?,PK?
0,0,id,INTEGER,1,1
1,1,timestamp,DATETIME,1,0
2,2,person_count,INTEGER,1,0
3,3,helmet_count,INTEGER,1,0
4,4,screenshot_path,VARCHAR,1,0


In [3]:
# Ganti 'violation' dengan nama tabel yang muncul di Cell 2 jika berbeda
# SQLModel biasanya menamai tabel lowercase dari nama class, jadi kemungkinan 'violation'
target_table = "violation" 

try:
    print(f"👀 5 Data Terakhir di tabel '{target_table}':")
    
    # Query ambil data
    df_sample = pd.read_sql(f"SELECT * FROM {target_table} ORDER BY id DESC LIMIT 5", conn)
    
    if df_sample.empty:
        print("⚠️ Tabel kosong, belum ada data pelanggaran.")
    else:
        # Tampilkan
        display(df_sample)
        
except Exception as e:
    print(f"Error: {e}")
    print("Mungkin nama tabel salah? Cek output Cell 2.")

👀 5 Data Terakhir di tabel 'violation':


,id,timestamp,person_count,helmet_count,screenshot_path
0,1,2025-12-04 02:10:09.534884,1,0,screenshots/vio_20251204_021009_ID1.jpg


In [4]:
# UNCOMMENT baris di bawah ini jika ingin menghapus data
# cursor = conn.cursor()
# cursor.execute("DELETE FROM violation;")
# conn.commit()
# print("⚠️ SELURUH DATA TELAH DIHAPUS!")

In [ ]:
import os
import glob

# 1. Tentukan folder target
folder_path = "screenshots"

# 2. Cek ketersediaan folder
if os.path.exists(folder_path):
    # Ambil list semua file di dalam folder tersebut
    files = glob.glob(os.path.join(folder_path, "*"))
    
    if len(files) == 0:
        print(f"ℹ️ Folder '{folder_path}' sudah bersih (kosong).")
    else:
        print(f"🗑️ Menghapus {len(files)} file di folder '{folder_path}'...")
        
        deleted_count = 0
        for file in files:
            try:
                os.remove(file)
                deleted_count += 1
            except Exception as e:
                print(f"❌ Gagal menghapus {file}: {e}")
        
        print(f"✅ SUKSES: {deleted_count} screenshot telah dihapus permanen.")
else:
    print(f"⚠️ Folder '{folder_path}' tidak ditemukan. Tidak ada yang dihapus.")

🗑️ Menghapus 85 file di folder 'screenshots'...
✅ SUKSES: 85 screenshot telah dihapus permanen.


: 